# Playing around with a basic blockchain

In [ ]:
import hashlib
import json
from typing import Any
from copy import deepcopy


class IntegrityError(RuntimeError):
    pass
       

In [ ]:
class BaseBlockChain:
    def __init__(self) -> None:
        self.__initial_block = {"hash": "initial", "block": []}
        self._chain: list[dict[str, Any]] = []

    @property
    def last_block(self) -> Any:
        if len(self._chain) < 1:
            return self.__initial_block
        else:
            return self._chain[-1]

    def __hash(self, previous_hash: str, block) -> str:
        new_block = dict(block=block)
        new_block["previous_hash"] = previous_hash

        new_block_string = json.dumps(new_block, sort_keys=True).encode()
        new_hash = hashlib.sha256(new_block_string).hexdigest()
        return new_hash
    
    def add_block(self, block: Any) -> None:
        previous_hash = self.last_block["hash"]
        new_block = {"hash": self.__hash(previous_hash, block), "block": block}
        self._chain.append(new_block)

    def check_integrity(self) -> None:
        i = 0
        previous_hash = self.__initial_block["hash"]
        for b in self._chain:
            if b["hash"] != self.__hash(previous_hash, b["block"]):
                raise IntegrityError(f"Invalid block in chain at position {i}: {b}")
            previous_hash = b["hash"]
            i += 1

    def print(self) -> None:
        i = 0
        for b in self._chain:
            print(f"{i}: {b}")
            i += 1


In [ ]:
c0 = BaseBlockChain()

c0.add_block("Block 1")
c0.add_block("Block 2")
c0.add_block("Block 3")
c0.check_integrity()

In [ ]:
c0.print()

In [ ]:
c1 = deepcopy(c0)
c1._chain[1]["block"] = "I tampered with you"
print("Print tampered version:")
c1.print()

print(" ")
print("Checking integrity:")
c1.check_integrity()

## A blockchain with transactions in bulk

In [ ]:
class BlockChainWithBulkTransactions(BaseBlockChain):
    def __init__(self) -> None:
        super().__init__()
        self._pending_transactions = []

    def add_transaction(self, transaction: Any) -> None:
        self._pending_transactions.append(transaction)    
    
    def commit(self) -> None:
        block = deepcopy(self._pending_transactions)
        self.add_block(block)
        self._pending_transactions = []

In [ ]:
ct0 = BlockChainWithBulkTransactions()

ct0.add_transaction("Block 1")
ct0.add_transaction("Block 2")
ct0.add_transaction("Block 3")
ct0.commit()
ct0.add_transaction("Block 4")
ct0.commit()
ct0.add_transaction("Block 5")
ct0.add_transaction("Block 6")
ct0.commit()
ct0.check_integrity()

ct0.print()

In [ ]:
ct1 = deepcopy(ct0)
ct1._chain[1]["block"] = "I tampered with you"
print("Print tampered version:")
ct1.print()

print(" ")
print("Checking integrity:")
ct1.check_integrity()